In [18]:
import numpy as np
import pandas as pd
import csv
from geopy.distance import vincenty
import random

In [20]:
plz_nrw = pd.read_csv('https://raw.githubusercontent.com/mexemt/location_optimization/master/Datasets/plz_nrw.csv', encoding='unicode_escape')
plz_nrw = pd.DataFrame(plz_nrw)
distances = pd.read_csv('https://raw.githubusercontent.com/mexemt/location_optimization/master/Datasets/distances.csv', encoding='unicode_escape', index_col=0)
distances = pd.DataFrame(distances)

In [22]:
distances

,52062,52064,52066,52068,52070,52072,52074,52076,52078,52080,...,42289,42327,42329,42349,42369,42389,42399,52146,46509,53909
plz,,,,,,,,,,,,,,,,,,,,,
52062,0.000000,0.833805,2.309969,2.543782,1.809500,5.355872,3.288065,9.066985,6.172449,5.368398,...,97.252918,85.152887,86.051710,89.802878,92.718592,99.817134,98.110042,6.071168,104.732184,43.253110
52064,0.833805,0.000000,2.005460,3.124071,2.641788,5.837448,2.731993,8.758703,6.367336,5.918150,...,97.995532,85.917421,86.804128,90.544667,93.447796,100.557649,98.830247,6.881911,105.530335,43.455263
52066,2.309969,2.005460,0.000000,2.760604,3.671349,7.665822,4.365852,6.781901,4.688917,5.043703,...,97.452614,85.497375,86.315558,89.999442,92.840291,100.003537,98.179906,7.100406,106.468564,41.649869
52068,2.543782,3.124071,2.760604,0.000000,2.159128,6.688759,5.810844,8.180905,4.101299,2.824678,...,94.956397,82.930060,83.786165,87.504288,90.385586,97.514427,95.754950,4.354309,103.855580,40.946118
52070,1.809500,2.641788,3.671349,2.159128,0.000000,4.538301,4.717025,10.053074,6.260380,4.613524,...,95.711995,83.556636,84.486637,88.264593,91.211795,98.281348,96.625293,4.393901,102.967750,42.997214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42389,99.817134,100.557649,100.003537,97.514427,98.281348,99.564296,102.996315,99.706324,96.206236,94.960017,...,2.620378,15.912863,14.079131,10.018650,7.626408,0.000000,5.305656,93.913405,77.342176,78.567689
42399,98.110042,98.830247,98.179906,95.754950,96.625293,98.131593,101.327213,97.546137,94.247215,93.140891,...,5.559837,16.843124,14.249761,9.726343,5.778035,5.305656,0.000000,92.288735,81.532872,74.925188
52146,6.071168,6.881911,7.100406,4.354309,4.393901,6.796071,9.104348,11.736866,6.652919,3.747935,...,91.341557,79.166390,80.105903,83.895772,86.860832,93.913405,92.288735,0.000000,99.593354,40.767530


In [24]:
locations = pd.DataFrame(plz_nrw['plz'])
customers = pd.DataFrame(plz_nrw['plz'])

In [ ]:
locations['open'] = 0
locations['fixed_costs'] = 0
customers['demand'] = 0

In [27]:
rng_cost_fx_low = range(10, 100)
rng_cost_fx_high = range(10, 100)
locations['fixed_costs'] = [random.choice(rng_cost_fx_low) for x in locations['fixed_costs']]

rng_demand_low = range(1, 10)
rng_demand_high = range(10, 100)
customers['demand'] = [random.choice(rng_demand_low) for x in customers['demand']]

In [29]:
customers

,plz,demand
0,52062,3
1,52064,1
2,52066,4
3,52068,8
4,52070,2
...,...,...
867,42389,4
868,42399,9
869,52146,4
870,46509,5


In [31]:
locations

,plz,open,fixed_costs
0,52062,0,66
1,52064,0,50
2,52066,0,70
3,52068,0,31
4,52070,0,75
...,...,...,...
867,42389,0,64
868,42399,0,93
869,52146,0,19
870,46509,0,62


In [33]:
distances

,52062,52064,52066,52068,52070,52072,52074,52076,52078,52080,...,42289,42327,42329,42349,42369,42389,42399,52146,46509,53909
plz,,,,,,,,,,,,,,,,,,,,,
52062,0.000000,0.833805,2.309969,2.543782,1.809500,5.355872,3.288065,9.066985,6.172449,5.368398,...,97.252918,85.152887,86.051710,89.802878,92.718592,99.817134,98.110042,6.071168,104.732184,43.253110
52064,0.833805,0.000000,2.005460,3.124071,2.641788,5.837448,2.731993,8.758703,6.367336,5.918150,...,97.995532,85.917421,86.804128,90.544667,93.447796,100.557649,98.830247,6.881911,105.530335,43.455263
52066,2.309969,2.005460,0.000000,2.760604,3.671349,7.665822,4.365852,6.781901,4.688917,5.043703,...,97.452614,85.497375,86.315558,89.999442,92.840291,100.003537,98.179906,7.100406,106.468564,41.649869
52068,2.543782,3.124071,2.760604,0.000000,2.159128,6.688759,5.810844,8.180905,4.101299,2.824678,...,94.956397,82.930060,83.786165,87.504288,90.385586,97.514427,95.754950,4.354309,103.855580,40.946118
52070,1.809500,2.641788,3.671349,2.159128,0.000000,4.538301,4.717025,10.053074,6.260380,4.613524,...,95.711995,83.556636,84.486637,88.264593,91.211795,98.281348,96.625293,4.393901,102.967750,42.997214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42389,99.817134,100.557649,100.003537,97.514427,98.281348,99.564296,102.996315,99.706324,96.206236,94.960017,...,2.620378,15.912863,14.079131,10.018650,7.626408,0.000000,5.305656,93.913405,77.342176,78.567689
42399,98.110042,98.830247,98.179906,95.754950,96.625293,98.131593,101.327213,97.546137,94.247215,93.140891,...,5.559837,16.843124,14.249761,9.726343,5.778035,5.305656,0.000000,92.288735,81.532872,74.925188
52146,6.071168,6.881911,7.100406,4.354309,4.393901,6.796071,9.104348,11.736866,6.652919,3.747935,...,91.341557,79.166390,80.105903,83.895772,86.860832,93.913405,92.288735,0.000000,99.593354,40.767530


## **Implementierung Add/Drop-Heuristik**

### Add-Heuristik

In [35]:
I = pd.DataFrame()

In [37]:
z_old = 1000000000000000
z_new = z_old-1
z_list = []
var_list = []
z_fx = 0

In [39]:
while z_new < z_old:
    z_old = z_new
    for index, row in locations.iterrows():
        if row['open'] == 0:
                z_fix = row['fixed_costs']
                z = z_fix + sum([a * b for a, b in zip(list(distances.iloc[len(range(index)),:]), list(customers.demand))])
                if z < z_new:
                    z_new = z
                    i_low = index
    if z_new < z_old:
        locations.ix[i_low]['open'] = 1
        z_fx = z_fx + z_fix
        I = I.append(distances.ix[locations.ix[i_low]['plz']], sort=False)

ToDo: Umschreiben der I = I.append(distances.ix[locations.ix[i_low]['plz']], sort=False) Funktion wegen pandas Update, Liste I mit kostenminimalen Lösungen vervollständigen

In [47]:
I

""


In [57]:
distances.loc[:,locations.loc[i_low,:]['plz']]

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [42555] of <class 'numpy.int64'>

In [54]:
locations.loc[i_low,:]['plz']

42555

In [45]:
pd.__version__

'1.0.3'